In [ ]:
#!/usr/bin/env python
# coding: utf-8


import numpy as np
# import matplotlib.pyplot as plt
import tensorflow as tf
import math

import time
import datetime as dt
from dateutil.parser import parse
import os
import pandas as pd
import openpyxl
import datetime as dt
import cx_Oracle as cx
import os
import warnings
warnings.filterwarnings('ignore')

ddi = 'C:/Users/junei/SODA/20211209_sugong/23개관측소/pre/'
dir_list = os.listdir(ddi)

# Hyperparameters
###
tf.set_random_seed()
n_pst = 
r_trn = 
# LTS parameters
n_lts_itr = 
###
r_lts = 
min_lv = 
r_lw = 
r_hg = 
# Optimization parameters
dec_step = 
###
n_itr = 
###
toler1 = 
###
toler2 =
# Design parameters
ref_int = 

def DFToDB(df):
    try:
        LOCATION = r"C:\Users\instantclient_21_3"
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록

        conn = cx.connect("", "", cx.makedsn("", "", ""))
        curs = conn.cursor()

        rows = [tuple(i) for i in df.to_records(index=False)]
        up_rows = []
        for j in df.index:
            r_1 = int(rows[j][0])
            r_2 = int(rows[j][1])
            r_3 = str(rows[j][2])
            r_4 = parse(rows[j][3])
            r_5 = int(rows[j][4])
            up_rows.append(tuple([r_1, r_2, r_3, r_4, r_5]))

        curs.executemany("INSERT INTO GD60 \
                         (A_ID, RESULT_ID, A_VALUE, \
                         R_DT, QLITY) \
                         VALUES (:1, :2, :3, :4, :5)", up_rows)
        conn.commit()
        curs.close()
        conn.close()
        print("DB Inserted")
        
    except Exception as e:
        print("데이터를 DB에 업로드하는 데 문제가 있습니다.\n", e)
        
def decode(Y_trn, Y_prd1, Y_tst, Y_prd2, n_trn):

    
def RNN_model(X):
    return out

def n_array(dat_res_raw, dat_exp_raw, dat_io):
                    
    Y_trn, Y_prd1, Y_tst, Y_prd2 = decode(Y_trn, Y_prd1, Y_tst, Y_prd2, n_trn)
    return Y_trn, Y_prd1, Y_tst, Y_prd2


# Normalization (0 ~ 1) of both explanatory and response data
def normalization(dat, r1, r2):
    minv = np.min(dat, axis=0)
    maxv = np.max(dat, axis=0)
    dat = (dat - minv) / (maxv - minv) * (r2 - r1)
    np.nan_to_num(dat, copy=False)
    dat = dat + r1
    return dat, minv, maxv


# LTS에 의한 비계절화와 추세 제거
def lts_preproc(dat, flg_season):

    return _Y, _X, _W

def process(df1, f):
    df1['일강수량(mm)'] = 0.0
    df1.reset_index(drop=True, inplace=True)

    try:
        dat_exp_raw1 = df1[['일강수량(mm)', '평균 상대습도(%)', '평균 현지기압(hPa)', '합계 일조 시간(hr)']].values
    except:
        dat_exp_raw1 = df1[['일강수량(mm)', '평균 상대습도(%)', '평균 현지기압(hPa)', '합계 일조시간(hr)']].values

    dat_res_raw1 = df1['수위'].values

    (일부 삭제)

    global _X, _W
    dat_res_raw, _X, _W = lts_preproc(dat_res_raw1, f)
    dat_exp_raw, _, _ = normalization(dat_exp_raw1, r_lw, r_hg)

    global minv_r, maxv_r
    dat_res_raw, minv_r, maxv_r = normalization(dat_res_raw, r_lw, r_hg)
    Y_trn, Y_prd1, Y_tst, Y_prd2 = n_array(dat_res_raw, dat_exp_raw, dat_io)

    # Save estimation results
    (삭제)

    return Y_prd

# 데이터 프레임 연 단위 분할
def mk_temp_year(temp_df):
    # 당년도 + 10일
    temp_df1 = temp_df[(temp_df.index <= pd.Timestamp(str(stand_date.year - 1)+'-01-10'))]
    temp_df.reset_index(drop=False, inplace=True)
    return temp_df

# 데이터 프레임 월 단위 분할
def mk_temp_month(temp_df):
    try:
        # 작년의 동일시기
        temp_df = temp_df[(temp_df.index > (stand_date - pd.Timedelta('365d') - pd.Timedelta('90d')))
                         & (temp_df.index <= (stand_date - pd.Timedelta('365d') + pd.Timedelta('100d')))]
        temp_df['일시'] = temp_df.index + pd.Timedelta('365d')

    except:
        # 재작년의 동일시기
        temp_df = temp_df[(temp_df.index > (stand_date - pd.Timedelta('730d') - pd.Timedelta('90d')))
             & (temp_df.index <= (stand_date - pd.Timedelta('730d') + pd.Timedelta('100d')))]
        temp_df['일시'] = temp_df.index + pd.Timedelta('730d')
       
    temp_df.reset_index(drop=True, inplace=True)
    return temp_df    

def up_data(temp_df, f, n):
    Y_prd = process(temp_df.copy(), f)
    # 고유번호에 맞는 ID값 매칭
    water_spot = pd.concat([temp_df['일시'].iloc[:-10], pd.DataFrame(Y_prd, columns=['수위'])], axis=1)
    my_tear = {'02' : 14789, '04' : 14995, '06' : 16218, '07' : 14717, '08' : 14700, '10' : 14687, 
               '11' : 14730, '12' : 16189, '14' : 14764, '22' : 14749, '24' : 16197, '25' : 14758}
    water_spot['A_ID'] = n
    water_spot['RESULT'] = my_tear['{0:02d}'.format(n)]
    water_spot['일시'] = water_spot['일시'].astype(str)
    water_spot['QLITY'] = 1 if len(water_spot) <= 300 else 2
    water_spot = water_spot[['A_ID', 'RESULT', '수위', '일시', 'QLITY']]
    tf.reset_default_graph()
    # 데이터 업로드
    return DFToDB(water_spot)

def start_pro(n):
    place = dir_list[n]
    print('='*10, place[:-9], '='*10)
    df = pd.read_csv((ddi + place), parse_dates=['일시'])
    
    global sess_save_nam, out_file_nam
    
    sess_save_nam = f'./save/{n}_rate8_Session_save_LSTM'
    
    df2 = df.copy()

    global strt_lr, r_dec, n_stat_rnn, n_lay, n_neu
    
    if n in [4, 6, 10]:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n == 12:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n == 14:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n == 2:
        f = 0
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n in [7, 8, 11]:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n == 22:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
    elif n in [24, 25]:
        f = 
        strt_lr = 
        r_dec = 
        n_stat_rnn = 
        n_lay = 
        
    n_neu = n_stat_rnn
    
    global stand_date
    # 오늘 날짜를 기준일로 설정
    stand_date = pd.Timestamp(dt.datetime.now().date())

    # 매년 1월 20일
    if (stand_date.month == 1) & (stand_date.day == 20):
        # 테스트 할 때 오늘 날짜 넣어서 하면 됨
#     if stand_date.day == 26:
        global sav_res
        sav_res = 1
        temp_df = df2.copy()
        temp_df.set_index('일시', inplace=True)
        up_data(mk_temp_year(temp_df), f, n)

        temp_df = df2.copy()
        temp_df.set_index('일시', inplace=True)
        up_data(mk_temp_month(temp_df), f, n)
        
    # 매월 15일
    elif stand_date.day == 15:
        # sav_res 0 먼저, 1 다음
        # 0일 때 불러와서 학습시키는 걸로...
        # sav_res = 0
        # 불러오기
        # temp_df = df2.copy()
#         temp_df.set_index('일시', inplace=True)
#         up_data(mk_temp_month(temp_df, f), n)

        temp_df = df2.copy()
        temp_df.set_index('일시', inplace=True)
        up_data(mk_temp_month(temp_df, f), n)
    
for spot in [2,]: # 4, 6, 7, 8, 10, 11, 12, 14, 22, 24, 25]:
    start_pro(spot)

C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:522: 

========== 02_양구방산_암반 ==========
INFO:tensorflow:Restoring parameters from ./save/2_rate8_Session_save_LSTM
The elapsed time is 0.09320378303527832 sec
INFO:tensorflow:Restoring parameters from ./save/2_rate8_Session_save_LSTM
The elapsed time is 0.08604693412780762 sec
========== 04_화천화천_암반 ==========
INFO:tensorflow:Restoring parameters from ./save/4_rate8_Session_save_LSTM
The elapsed time is 0.11367940902709961 sec
INFO:tensorflow:Restoring parameters from ./save/4_rate8_Session_save_LSTM
The elapsed time is 0.10123801231384277 sec
========== 06_양평양동_충적 ==========
INFO:tensorflow:Restoring parameters from ./save/6_rate8_Session_save_LSTM
The elapsed time is 0.13370037078857422 sec
INFO:tensorflow:Restoring parameters from ./save/6_rate8_Session_save_LSTM
The elapsed time is 0.12540411949157715 sec
========== 07_평택안중_암반 ==========
INFO:tensorflow:Restoring parameters from ./save/7_rate8_Session_save_LSTM
The elapsed time is 0.13495492935180664 sec
INFO:tensorflow:Restoring parameter